In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab10.ipynb")

<table style="width: 100%;">
    <tr style="background-color: transparent;"><td>
        <img src="https://data-88e.github.io/assets/images/blue_text.png" width="250px" style="margin-left: 0;" />
    </td><td>
        <p style="text-align: right; font-size: 10pt;"><strong>Economic Models</strong>, Fall 2021<br>
            Dr. Eric Van Dusen<br>
            Alan Liang<br>
        Umar Maniku<br>
        </p></td></tr>
</table>

# Lab 10: Consumer Spending and Economic Stimulus Payments

In [43]:
import warnings
warnings.simplefilter("ignore")
from datascience import *
import statsmodels.api as sm
import numpy as np
import nbforms
# feedback = nbforms.Form("feedback_form_config.json")

def get_dummies(tbl, col, drop=True, drop_first=True):
    """Creates dummy variables for a column of a table"""
    values = np.unique(tbl.column(col))
    if drop_first:
        values = values[1:]
    for val in values:
        encoding = tbl.apply(lambda s: int(s == val), col)
        tbl = tbl.with_column(col + "_" + str(val), encoding)
    if drop:
        tbl = tbl.drop(col)
    return tbl

Welcome to lab 10! 

In this lab, you will investigate how economic stimulus payments in the form of tax rebates affect household consumption. This lab is based on *Household Expenditure and the Income Tax Rebates of 2001* by David S. Johnson, Jonathan A Parker, and Nicholas S Souleles (which we will refer to as the JPS study for short). 

In 2001, the Bush administration passed the *Economic Growth and Tax Relief Reconciliation Act of 2001*, which mainly  reduced the rates of individual income taxes. In addition, the bill initiated a series of one-time rebates for all taxpayers that filed a tax return for 2000. The payment of these rebates were broadly announced, so that most households were aware of an incoming stimulus payment (much similar to the recent stimulus check). The rebate was as follows:
- Up to a maximum of \$300 for single filers with no dependents
- Up to a maximum of \$500 for single parents
- Up to a maximum of \$600 for married couples

We are interested in determining how individuals altered their consumption patterns due to the economic stimulus payments, and by extent see if the permanent income hypothesis holds. The permanent income hypothesis states that consumers attempt to smooth their consumption across their life time, so that "changes in permanent income, rather than changes in temporary income, are what drive the changes in a consumer's consumption patterns." Intuitively, if the permanent income hypothesis were to hold, we would expect households to smooth out the spending of the rebate even before the rebate arrived. Thus, their consunmption would not change much between periods since they had known in the previous period that they would be receiving a sizable increase in income in the near future.

Notably, these stimulus payments were assigned to households at random periods in time, which allows us to better conclude a causal effect of a one time cash payment on changes in consumption.

To determine the true causal effect, we will use least squares regression. JPS propose the following regression for any household relating the stimulus payment to change in consumption:

$$C_{t+1} - C_t = \sum_s \beta_{0,s} \text{month}_s + \beta_1 \text{age} + \beta_2 \text{$\Delta$ children} + \beta_3 \text{$\Delta$ adults} + \beta_4 \text{Stimulus Payment}_{t+1} + u$$

Here, we control for seasonal effects by creating dummy variables for each period (measured in months), and also control for changes in the number of children and adults in a household. Let's consider a few scenarios in context of the regression to gain some intuition:
- If a household received a stimulus in period $t+1$, then the change in consumption ($C_{t+1} - C_t$) due to the rebate should be captured by $\beta_4$ if we have sufficiently controlled for all potential factors of change in consumption between the 2 periods. 
- If a household did not receive a stimulus payment in $t+1$, then the stimulus payment will be 0. Thus, the change in consumption will only be explained by our control variables: age, changes in family members, and seasonal variation. 



Let's read in the table. The columns labels are:

| Label | Description |
|------|------|
| `newid` | household ID |
| `year_month` | month when data was collected |
| `dcf` | change in food expenditures |
| `dcs` | change in strictly nondurable expenditures |
| `dcn` | change in nondurable expenditures |
| `dlcf` | change in log food expenditures |
| `dlcs` | change in log strictly nondurable expenditures |
| `dlcn` | change in log nondurable expenditures |
| `dnumadult` | change in number of adults |
| `dnumkids` | change in number of kids |
| `age` | average age of head & spouse (if exists) |
| `taxreb` | total rebates received in reference period |
| `ltaxreb` | rebates received in prior reference period (-1) |
| `l2taxreb` | rebates received in twice prior reference period (-2) |

In [2]:
rebates = Table.read_table("JPS.csv")
rebates

One very important thing to note is that the unit of observation is not per household, but rather per time period per household. If a household were observed at 3 different time periods, then they would make up 3 rows and hence "contribute 3 times to the regression". This kind of set up is most oftne referred to as a *panel data study*. 

Let's visualize the data. Below is a household that received a stimulus payment in August 2001.

In [3]:
rebates.where('newid', 116249)

Thus for the data point in which $t+1$ refers to August (so that $t$ refers to May), $\text{Stimulus Payment}_{t+1} = 120$. For the data point in which $t+1$ refers to November (so that $t$ refers to August), $\text{Stimulus Payment}_{t+1} = 0$ and $\text{Stimulus Payment}_t = 120$. 

In general, we will use `taxreb` as the $\text{Stimulus Payment}_{t+1}$ variable.

### Question 1
Let's try to recreate JPS' regression. We have selected the relevant columns for the independent variables:

In [4]:
X_q1 = rebates.select("year_month", "dnumadult", "dnumkids", "age", "taxreb")
X_q1

**Question 1.1:** Create dummy variables to represent the different months. Augment the `X_q1` table with dummy variables for `year_month`, and assign it to `X_q1_dummies`.

<!--
BEGIN QUESTION
name: q1_1
-->

In [5]:
X_q1_dummies = ...
X_q1_dummies

In [ ]:
grader.check("q1_1")

**Question 1.2:** Conduct an OLS regression of change in food consumption using `statsmodels` replicating JPS' setup. Interpret the coefficient on `taxreb`.

<!--
BEGIN QUESTION
name: q1_2
-->

In [7]:
q1_2_X = ...
q1_2_y = ...
model_q1_2 = ...
model_q1_2.summary()

In [ ]:
grader.check("q1_2")

**Question 1.3:** Conduct an OLS regression of change in consumption for strictly non-durable goods using `statsmodels` replicating JPS' setup. How does the coefficient on `taxreb` compare with that for 1.2?

<!--
BEGIN QUESTION
name: q1_3
-->

In [12]:
q1_3_X = ...
q1_3_y = ...
model_q1_3 = ...
model_q1_3.summary()

In [ ]:
grader.check("q1_3")

### Question 2

For the second part, we will treat the variable $\text{Stimulus Payment}$ as a binary variable, in which $\text{Stimulus Payment}_{t+1} = 1$ if the household received a stimulus payment in period $t+1$, and $\text{Stimulus Payment}_{t+1} = 0$ if not. 

In [17]:
X_q2 = rebates.select("year_month", "dnumadult", "dnumkids", "age", "taxreb")
X_q2

**Question 2.1:** Create a binary variable to represent whether a stimulus payment was received and add it to `X_q2_1` as a column called `itaxreb`. Make sure to drop `taxreb`.

<!--
BEGIN QUESTION
name: q2_1
-->

In [18]:
X_q2_1 = ...
X_q2_1

In [ ]:
grader.check("q2_1")

**Question 2.2:** Similar to 1.1, create dummy variables to represent the different months. Augment the `X_q2_1` table with dummy variables for `year_month`, and assign it to `X_q2_dummies`.

<!--
BEGIN QUESTION
name: q2_2
-->

In [22]:
X_q2_dummies = ...
X_q2_dummies

In [ ]:
grader.check("q2_2")

**Question 2.3:** Conduct an OLS regression of change in food consumption using `statsmodels` replicating JPS' setup. Interpret the coefficient on `itaxreb`, and compare this with your results in 1.2.

<!--
BEGIN QUESTION
name: q2_3
-->

In [24]:
q2_3_X = ...
q2_3_y = ...
model_q2_3 = ...
model_q2_3.summary()

In [ ]:
grader.check("q2_3")

**Question 2.4:** Conduct a simliar OLS regression of change in strictly non-durable consumption. How does the coefficient on `itaxreb` compare with your results in 1.3?

<!--
BEGIN QUESTION
name: q2_4
-->

In [28]:
q2_4_X = ...
q2_4_y = ...
model_q2_4 = ...
model_q2_4.summary()

In [ ]:
grader.check("q2_4")

<!-- BEGIN QUESTION -->

**Question 2.5:** What are the differences in consumption changes between food and strictly non-durables?

<!--
BEGIN QUESTION
name: q2_5
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Question 2.6:** Looking at non-durables is more relevant in the context of the permanent income hypothesis. Strictly non-durable goods like food do not last between time periods, so that households consume the good in 1 period only. Thus, we can attribute a change in non-durable consumption to consumption that was actually carried out in the corresponding period.

What would we expect $\beta_4$ to be if the permanent income hypothesis were to hold? Is this result in line with your OLS results from question 1 and 2?

<!--
BEGIN QUESTION
name: q2_6
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



### Question 3

One potential issue with the JPS study is that households who received a period in period $t$ but not in $t+1$ will have the same $\text{Stimulus Payment}_{t+1}$ value as households who did not receive a stimulus payment at all. This may not be the case - intuitively we might expect consumption to decrease if a payment was issued in the previous period but not the current period. Instead, we will implement an added variable on whether a household received stimulus payment in period $t$ to better control for the causal effect of a stimulus payment on changes in consumption. 

We will augment the JPS setup with an added variable $\text{Stimulus Payment}_{t}$:
$$C_{t+1} - C_t = \sum_s \beta_{0,s} \text{month}_s + \beta_1 \text{age} + \beta_2 \text{$\Delta$ children} + \beta_3 \text{$\Delta$ adults} + \beta_4 \text{Stimulus Payment}_{t+1} + \beta_5 \text{Stimulus Payment}_{t} + u$$

Thus:
- If a household received a stimulus in period $t+1$ only, then the change in consumption ($C_{t+1} - C_t$) due to the rebate should be captured by $\beta_4$ if we have sufficiently controlled for all potential factors of change between the 2 periods. 
- If a household did not receive a stimulus payment in $t+1$ or $t$, then both stimulus payment variables will be 0. Thus, the change in consumption will only be explained by our control variables: age, changes in family members, and seasonal variation. 
- If a household received a stimulus in period $t$ only, then the change in consumption ($C_{t+1} - C_t$) due to the rebate should be captured by $\beta_5$ if we have sufficiently controlled for all potential factors of change between the 2 periods. 

Notably, interpreting the coefficient $\beta_5$ for $\text{Stimulus Payment}_{t}$ will allow us to determine how much consumption will change in the period after a household receives the stimulus check.


The columns `ltaxreb` reflect the stimulus payment received in the previous period.


In [32]:
X_q3 = rebates.select("year_month", "dnumadult", "dnumkids", "age", "taxreb", "ltaxreb")
X_q3

**Question 3.1:** Conduct a new regression of change in food consumption using the new regression model proposed in question 3. Interpret $\beta_4$ and $\beta_5$. Do your results differ much from that of question 1?

<!--
BEGIN QUESTION
name: q3_1
-->

In [33]:
q3_1_X = ...
q3_1_y = ...
model_q3_1 = ...
model_q3_1.summary()

In [ ]:
grader.check("q3_1")

**Question 3.2:** Conduct a new regression of change in strictly non-durable consumption using the new regression model proposed in question 3. Interpret $\beta_4$ and $\beta_5$. Do your results differ much from that of question 1?

<!--
BEGIN QUESTION
name: q3_2
-->

In [37]:
q3_2_X = ...
q3_2_y = ...
model_q3_2 = ...
model_q3_2.summary()

In [ ]:
grader.check("q3_2")

In [41]:
# Just run the next few cells.
X_q3_3 = X_q3.with_columns("itaxreb", (X_q3.column("taxreb") > 0).astype(int),
                           "iltaxreb", (X_q3.column("ltaxreb") > 0).astype(int)).drop('taxreb', 'ltaxreb')
q3_3_X = get_dummies(X_q3_3, "year_month").to_df()
q3_3_y = rebates.column("dcf")
model_q3_3 = sm.OLS(q3_3_y, sm.add_constant(q3_3_X)).fit()
model_q3_3.summary()

In [42]:
q3_4_X = q3_3_X
q3_4_y = rebates.column("dcs")
model_q3_4 = sm.OLS(q3_4_y, sm.add_constant(q3_4_X)).fit()
model_q3_4.summary()

<!-- BEGIN QUESTION -->

**Question 3.3:**  We have repeated the regressions of change in food consumption and change in strictly non-durable consumption above, but with the stimulus payment as a binary variable, like in question 2. Interpret $\beta_4$ and $\beta_5$ in both regressions. Do your results differ much from that of question 2?

<!--
BEGIN QUESTION
name: q3_3
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



### An Afterword

Throughout this study, we could have made many other adjustments to our regression. For example, we could have determined changes in log consumption, controlled for more variables, or only regressed on households that received a stimulus payment. In this lab, we only tried changing how we captured the stimulus payment: as a dollar amount and as a binary variable, and the results were somewhat different.

As you can see, doing econometrics often relies on many value judgments. Each subtle decision you make on your data or model may lead to large changes in your regression outcomes, and could be the difference between statistical significance and insignificance.

Something to keep in mind as we conduct many models to try out different adjustments is to be aware of potential p-hacking. With a p-value of 5%, we would expect to see statistically significant results even if the null hypothesis were true 1 in 20 times. Thus, even if the null hypothesis were true, a model may produce statistically significant results if we ran enough variations of the model.

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)